In [25]:
import DataConnection as Dc
from collections import defaultdict
import time
import datetime
import operator
import csv
import matplotlib 



pgo = Dc.PGConnection('CRASHDATA', 'shostetter')  #Database Connection
dbo = Dc.DBConnection('dotgissql01', 'gisgrid', 'GISUSER', 'GISUSER') #Database Connection
dbo2 = Dc.DBConnection('DOTDEV55SQL02', 'DataWarehouse', 'SHostetter', 'shostetter') #Database Connection



def get_intersection_universe(pg):
    cur = pg.conn.cursor()
    #.conn.cursor() Handles the connection to a PostgreSQL database instance. It encapsulates a database session. Return a new cursor object using the connection.

    cur.execute("select nodeid, masterid, is_int  from node where is_int = true")#Execute a database operation (query or command)
    #print(cur.fetchall())
    # dictionary of nodeid: [masterid, intersection]
    nodes = defaultdict(list, ((int(row[0]), [row[1], row[2]]) for row in cur.fetchall()))
    # cur.fetchall() Fetch all (remaining) rows of a query result, returning them as a list of tuples. An empty list is returned if there is no more record to fetch.
    del cur
    #deletes cur to reset the memory for a new query
    return nodes



def get_unsignalized_masterids(nodes):
    mids = defaultdict(list)
    to_remove = set()
    for node in nodes.keys():
        mids[nodes[node][0]].append(node)
        if nodes[node][-1] == 'Signal':
            to_remove.add(nodes[node][0])
    for m in to_remove:
        del(mids[m])
    return mids

ImportError: cannot import name _distributor_init

In [5]:
node_dict = get_intersection_universe(pgo)

In [6]:
node_dict


defaultdict(list,
            {9043971: [9043971, True],
             14: [14, True],
             18: [18, True],
             19: [19, True],
             22: [22, True],
             23: [23, True],
             25: [25, True],
             26: [26, True],
             27: [27, True],
             9043999: [9044000, True],
             9044000: [9044000, True],
             9044003: [9044003, True],
             9044004: [9044004, True],
             47: [47, True],
             51: [51, True],
             52: [52, True],
             53: [53, True],
             54: [54, True],
             55: [55, True],
             56: [56, True],
             57: [57, True],
             59: [59, True],
             62: [62, True],
             63: [63, True],
             64: [64, True],
             65: [65, True],
             66: [66, True],
             67: [67, True],
             68: [68, True],
             69: [69, True],
             70: [70, True],
             71: [71, True],
    

In [9]:
def alt_get_signalized_int(db, pg, nodes):
    cur = db.conn.cursor()
    cur.execute("""SELECT [PSGM_ID],[ContrType],[NodeID],[Longitude],[Latitude]
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    --[LION_CURRENT].[GISADMIN].[SIG_TRpsgm] 
                    where NormalizedType != 'Z'
                    --and [Longitude] not in ('','S' ) and  [Latitude] not in ('')
                    and [Longitude] is not null
                    --and psgm_id in (41084, 41083)""")
    #for row in cur .fetchall():
    #    psgm_id, contrtype, nodeid, x, y = row
        #print('****',psgm_id, contrtype, nodeid, x, y,'*********')
        nodeid = alt_get_node_from_signal_coords(pg, str(x), str(y))
        #print('-----------',nodeid,'--------------')
              
        if nodeid:
            nodes[int(nodeid[0])].append([nodeid[1],'Signal'])
    del cur
    return nodes


def alt_get_node_from_signal_coords(pg, x, y):
    cur = pg.conn.cursor()
    cur.execute("""select nodeid, st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) as distance
                    from node where st_dwithin(st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263),geom, 1000)
                    and is_int = true
                    order by st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) ASC limit 1
                """ % (x, y, x, y, x, y)
                )
    row = cur.fetchone()
    del cur
    if row:
        return int(row[0]),row[1]
    else:
        return None
# make list of unsignalized masterids

In [10]:
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
node_dict2 = alt_get_signalized_int(dbo, pgo, node_dict)
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

2019-07-11 18:05:07
2019-07-11 18:11:55


In [11]:
node_dict2

defaultdict(list,
            {9043971: [9043971, True],
             14: [14, True],
             18: [18, True],
             19: [19, True],
             22: [22, True],
             23: [23, True],
             25: [25, True],
             26: [26, True],
             27: [27, True],
             9043999: [9044000, True],
             9044000: [9044000, True, [25.6772516852635, 'Signal']],
             9044003: [9044003, True],
             9044004: [9044004, True],
             47: [47, True],
             51: [51, True, [12.5625710048057, 'Signal']],
             52: [52, True],
             53: [53, True],
             54: [54, True],
             55: [55, True],
             56: [56, True],
             57: [57, True],
             59: [59, True],
             62: [62, True],
             63: [63, True],
             64: [64, True],
             65: [65, True],
             66: [66, True],
             67: [67, True],
             68: [68, True],
             69: [69, True],
  

In [12]:
mx=[]
for i in node_dict2.keys():
    #print(i)
    if i==0 or i==1:
        pass
    elif node_dict2[i][-1] ==True:
            pass
    else:
        mx.append(node_dict2[i][-1][0])

In [19]:
mx.sort(reverse=1)

None


In [21]:
plot(mx)

NameError: name 'plot' is not defined

In [105]:
print node_dict2[51]
print(type(node_dict2[9043971][-1]))

[51, True, 'Signal', 'Signal', [12.5625710048057, 'Signal']]
<type 'bool'>


[992.867900320983,
 919.095497940198,
 861.284538139828,
 854.697638760513,
 817.044888199981,
 811.246126682375,
 804.869590665459,
 746.530336457781,
 739.937220732902,
 724.02721277857,
 681.014338286429,
 679.814197394541,
 642.173065202711,
 604.528574390688,
 598.503995585278,
 587.537715971367,
 585.407881076897,
 584.957247385563,
 581.457721125198,
 577.418926959424,
 572.14565107034,
 563.651420285417,
 556.64172337328,
 550.658293158952,
 549.380512322983,
 544.498856023029,
 540.813881944749,
 531.299347173286,
 520.470384332296,
 520.193122247388,
 519.110737293575,
 513.137241972542,
 512.042736580082,
 506.668262454909,
 502.413262402798,
 500.929590247364,
 494.742792969751,
 489.295514210853,
 488.075100664097,
 485.548048691083,
 476.750091753032,
 473.330408335881,
 472.597911927904,
 469.97037422584,
 466.898742703095,
 459.058899989734,
 458.329880758915,
 455.773824789468,
 454.893331847083,
 446.54824458282,
 446.303618964655,
 445.561850455824,
 445.355345362685

In [132]:
mx.sort(reverse=True)

In [133]:
print(mx)

[992.867900320983, 919.095497940198, 861.284538139828, 854.697638760513, 817.044888199981, 811.246126682375, 804.869590665459, 746.530336457781, 739.937220732902, 724.02721277857, 681.014338286429, 679.814197394541, 642.173065202711, 604.528574390688, 598.503995585278, 587.537715971367, 585.407881076897, 584.957247385563, 581.457721125198, 577.418926959424, 572.14565107034, 563.651420285417, 556.64172337328, 550.658293158952, 549.380512322983, 544.498856023029, 540.813881944749, 531.299347173286, 520.470384332296, 520.193122247388, 519.110737293575, 513.137241972542, 512.042736580082, 506.668262454909, 502.413262402798, 500.929590247364, 494.742792969751, 489.295514210853, 488.075100664097, 485.548048691083, 476.750091753032, 473.330408335881, 472.597911927904, 469.97037422584, 466.898742703095, 459.058899989734, 458.329880758915, 455.773824789468, 454.893331847083, 446.54824458282, 446.303618964655, 445.561850455824, 445.355345362685, 444.580705492128, 442.410805591575, 440.6888987853

In [35]:
node_dict2.keys()[2]



9043971

In [ ]:
masters = get_unsignalized_masterids(node_dict)

In [ ]:
masters

In [124]:
def get_all_crashes_nypd(db2):
    # test speed of doing 1 query and iterating in python memory
    cur = db2.conn.cursor()
    cur.execute( """SELECT distinct c.[ANUM_PCT]+c.[ANUM_YY]+c.[ANUM_SEQ] as crashid ,
                                    c.[NODEID],  cast(c.[OCCURRENCE_DATETIME] as DATE) as accd_date,  vic.VICTIM_NUMBER,
                                    c.[INJURED_COUNT],c.[KILLED_COUNT],  
                                    case when vic.ped_nonped = 'P' then 1
                                    when vic.PED_NONPED = 'B' then 2 else 3 end as inj_mode,
                                    c.[ACCIDENT_DIAGRAM]  AS [COLLISION_TYP],
                                    null as ext_of_inj, 0 as pre_action, 0 as direction, 0 as contrib
                    FROM [DataWarehouse].[dbo].[AIS_PD_Core_F] AS c
                    LEFT OUTER JOIN (
                            --selects crashid breakdown, and non preventable accident actions 
                            SELECT [ANUM_PCT], [ANUM_YY], [ANUM_SEQ],
                                    MAX(CASE WHEN PRE_ACDNT_ACTION = '10' THEN 1
                                             WHEN PRE_ACDNT_ACTION in ('15', '08', '09', '14', '04') THEN 1 -- removes other non-preventable
                                             ELSE 0 END) AS PK
                            FROM [DataWarehouse].dbo.AIS_PD_Vehicle_F
                            GROUP BY [ANUM_PCT], [ANUM_YY], [ANUM_SEQ]
                            )AS v -- parked vehicles
                            ON c.[ANUM_PCT] = v.ANUM_PCT AND c.[ANUM_YY] = v.ANUM_YY AND c.[ANUM_SEQ] = v.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Victim_F as vic --Victim Table
                            ON c.ANUM_PCT = vic.ANUM_PCT and c.ANUM_SEQ = vic.ANUM_SEQ and c.ANUM_YY = vic.ANUM_YY
                    WHERE C.OCCURRENCE_DATETIME between '3/24/2016' and '10/20/2016'
                    --and C.NODEID != 0
                    and c.NODEID = 33755
                    and v.PK != 1
                    --and vic.INJURED_COUNT >0
                """)
    data = cur.fetchall()
    del cur
    # clean up dates
    for row in data:
        d = row[2]  # date field
        row[2] = datetime.datetime.strptime(d, "%Y-%m-%d").date()
    assert isinstance(data, object)
    return data

In [ ]:
crash_list = get_all_crashes_nypd(dbo2)  # NYPD data

In [126]:
print(len(crash_list))

24


In [188]:
print 'NodeID: ', crash_list[11]

NodeID:  (u'1061610582', 33755, datetime.date(2016, 4, 30), 3, 3, 0, 3, u'4', None, 0, 0, 0)


In [191]:
for i in range(len(crash_list)):
    print 'NodeID: ', crash_list[i][1], '| CrashID: ', crash_list[i][0], '| Collision Type: ', crash_list[i][7]

NodeID:  33755 | CrashID:  1061610117 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610251 | Collision Type:  2
NodeID:  33755 | CrashID:  1061610293 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610326 | Collision Type:  9
NodeID:  33755 | CrashID:  1061610334 | Collision Type:  7
NodeID:  33755 | CrashID:  1061610343 | Collision Type:  2
NodeID:  33755 | CrashID:  1061610389 | Collision Type:  1
NodeID:  33755 | CrashID:  1061610431 | Collision Type:  7
NodeID:  33755 | CrashID:  1061610523 | Collision Type:  9
NodeID:  33755 | CrashID:  1061610582 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610582 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610582 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610741 | Collision Type:  4
NodeID:  33755 | CrashID:  1061610840 | Collision Type:  1
NodeID:  33755 | CrashID:  1061610948 | Collision Type:  4
NodeID:  33755 | CrashID:  1061611237 | Collision Type:  1
NodeID:  33755 | CrashID:  1061611268 | Collision Type: 

def get_all_dir_right_angle_crashes_nypd(db2):
    cur = db2.conn.cursor()
    cur.execute("""/* conservative */
                    -- north / south
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                        and PRE_ACDNT_ACTION != '10'
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3, 7)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- east / west
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3,7)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northeast / southwest
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northwest / southeast
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    ra = cur.fetchall()
    del cur
    return set([i[0] for i in ra])

In [72]:
drac = get_all_dir_right_angle_crashes_nypd(dbo2)  # NYPD data

In [ ]:
for i in drac: print(int(i))

In [171]:
count=[]
for i in range(len(crash_list)):
    #print crash_list[i][0]
    if crash_list[i][0] in drac:
            
            count.append(crash_list[i][0])
            
for i in count :
    print(i)
    
p = (set(count))
print(p)

1061610293
1061610582
1061610582
1061610582
1061610741
1061612291
1061612306
1061612652
set([u'1061612652', u'1061610741', u'1061610293', u'1061610582', u'1061612306', u'1061612291'])


In [130]:
class Intersection:
    def __init__(self, masterid, nodeid_list, crashes_list):
        self.masterid = masterid
        self.nodeid_list = nodeid_list
        self.crashes = crashes_list
        self.one_year_crashes = dict()
        self.twelve_month_period()
        self.warrant()
        self.get_12_month_crashes()

    def five_or_more(self):
        count = 0
        for c in self.crashes:
            if c.preventable():
                count += 1
        if count > 4:
            return True
        else:
            return False

    def twelve_month_period(self):
        cl = [[c.crashid, c.ac_date] for c in self.crashes if c.preventable()]
        cl = sorted(cl, key=operator.itemgetter(1), reverse=True)
        while len(cl) > 1:
            if (cl[0][1] - cl[-1][1]).days / 365.0 > 1:
                cl.pop()
            else:
                break
        return cl

    def warrant(self):
        if len(self.twelve_month_period()) > 4:
            return True
        else:
            return False

    def get_12_month_crashes(self):
        crashes = dict((c[0], None) for c in self.twelve_month_period())
        for crash in self.crashes:
            if crash.crashid in crashes.keys():
                crashes[crash.crashid] = crash
        return crashes


In [131]:
def add_intersection_objects_to_master_dict(mids):
    for m in mids:
        mids[m].append(Intersection(m, mids[m], list()))
        if isinstance(mids[m][-1].nodeid_list[-1], Intersection):
            mids[m][-1].nodeid_list = mids[m][-1].nodeid_list[:-1]
            


In [132]:
add_intersection_objects_to_master_dict(masters)

In [208]:
masters.five_or_more(33755)
#for i in masters:
 #   print(i)
  #  if i == 33755:
        

AttributeError: 'collections.defaultdict' object has no attribute 'five_or_more'

In [146]:
class Crash:
    def __init__(self, crashid, nodeid, ac_date, vic_number, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac):
        """
        :param crashid:
        :param nodeid:
        :param ac_date:
        :param vic_number:
        :param inj:
        :param fat:
        :param mode:
        :param col_typ:
        :param sev:
        :param pre_actions:
        :param directions:
        :param contrib_fac:
        :return:
        """
        self.crashid = crashid
        self.nodeid = nodeid
        self.inj = inj
        self.fat = fat
        self.mode = mode
        self.col_typ = str(col_typ)
        self.pre_actions = pre_actions
        self.directions = directions
        self.ac_date = ac_date
        self.vic_number=vic_number
        self.sev = sev
        self.contib_fac = contrib_fac
        self.right_angle()
        self.ped_bike()
        self.preventable()

    def right_angle(self):
        """
        includes: right angle (4), left turn with (0), and right turn with (5)
        :return: Boolean
        """
        if self.col_typ in ('4', '0', '5'):  # NYPD
            return True
        elif self.col_typ in ('04', '10', '05'):  # NYSDOT
            return True
        else:
            return False

    def ped_bike(self):
        if self.mode in (1, 2):
            return True
        else:
            return False

    def directions_prevent(self):
        if self.directions:
            return True
        else:
            return False

    def preventable(self):
        if self.right_angle() or self.ped_bike() or self.directions_prevent():
            if self.directions_prevent() and not self.right_angle():
                print '%s RA (%s), PB (%s), DIR(%s)' % (str(self.crashid),
                                                    str(self.right_angle()),
                                                    str(self.ped_bike()),
                                                    str(self.directions_prevent())
                                                    )
            return True
        else:
            return False

In [154]:
def add_crashes_to_intersections(nodes, mids, crashes, drac):
    for crash in crashes:
        crashid, nodeid, ac_date, vic_number, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac = crash
        if crashid in drac:
            directions = True
        else:
            directions = False
        if nodes[int(nodeid)]:
            master = nodes[int(nodeid)][0]
            if master in mids.keys():  # check for unsignalized
                mids[master][-1].crashes.append(Crash(crashid, nodeid, ac_date, vic_number, inj, fat, mode,
                                                      col_typ, sev, pre_actions, directions, contrib_fac))

In [155]:
add_crashes_to_intersections(node_dict, masters, crash_list, drac)


1061612291 RA (False), PB (False), DIR(True)
1061612652 RA (False), PB (False), DIR(True)


In [156]:
def approved_intersections(md):
    ints_w_warrant_met = dict()
    for master in md.keys():
        if md[master][-1].warrant():
            ints_w_warrant_met[master] = md[master][-1].get_12_month_crashes()
            #print master, len(md[master][-1].twelve_month_period())
    return ints_w_warrant_met

In [157]:
approved_masters = approved_intersections(masters)

1061612291 RA (False), PB (False), DIR(True)
1061612652 RA (False), PB (False), DIR(True)
1061612291 RA (False), PB (False), DIR(True)
1061612652 RA (False), PB (False), DIR(True)


In [175]:
#print(approved_masters)
print(len(approved_masters[33755]))
for i in approved_masters[33755]:
    print(i)

12
1061612384
1061612537
1061612652
1061610523
1061610326
1061610741
1061610948
1061610293
1061612306
1061610117
1061610582
1061612291


In [159]:
def get_node_details(pg):
    data = dict()
    cur = pg.conn.cursor()
    cur.execute("""select  nodeid, masterid, st1, st2, st_x(geom) as x, st_y(geom) as y
                    from node n
                    join (
                        select nodeidfrom, min(st1) as st1, max(st2) as st2
                        from(
                            select s1.nodeidfrom, s1.street as st1, s2.street  as st2
                            from lion as s1 join lion as s2
                            on s1.nodeidfrom = s2.nodeidto
                            where s1.street != s2.street
                            and s1.featuretyp not in ('1', '2', '3', '7')
                            and s2.featuretyp not in ('1', '2', '3', '7')
                        ) as names_ group by nodeidfrom
                    ) as st_names on n.nodeid::int = st_names.nodeidfrom::int
                """)
    for row in cur.fetchall():
        nodeid, masterid, st1, st2, x, y = row
        data[nodeid] = [st1, st2, x, y]
    del cur
    return data

node_data = get_node_details(pgo)
node_data

In [209]:
for i in node_data:
    if i == 33755:
     print node_data[i]

['ROCKAWAY BOULEVARD', '110 STREET', 1031726.09807865, 186224.401813105]


In [160]:
def intersection_summaries(approved, mids, nds):
    summary = list()
    for m in approved.keys():
        crashes = len([c for c in approved[m]])
        ped_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 1])
        bike_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 2])
        mvo_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 3])
        right_angles = len([approved[m][c].crashid for c in approved[m] if approved[m][c].col_typ in ('4', '04')])
        date_ranges = [min([approved[m][c].ac_date for c in approved[m]]),
                       max([approved[m][c].ac_date for c in approved[m]])]
        print 'Master %i (%s - %s):\n\tNodes %s \n\t%i crashes\n\t%i ped injuries\n\t%i bike' \
              'injuries\n\t%i mvo injuries\n\t%i right angle crashes' % (m, date_ranges[0].strftime('%m/%d/%Y'),
                                                                         date_ranges[1].strftime('%m/%d/%Y'),
                                                                         str(mids[m][-1].nodeid_list),
                                                                         crashes, ped_inj, bike_inj, mvo_inj,
                                                                         right_angles)
        if len(mids[m]) > 2:
            for node in mids[m][:-1]:
                if node in nds.keys():
                    s1, s2, x, y = nds[node]
                else:
                    s1, s2, x, y = '', '', 0, 0
                summary.append([m, node, crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                                right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
        else:
            if mids[m][0] in nds.keys():
                s1, s2, x, y = nds[mids[m][0]]
            else:
                s1, s2, x, y = '', '', 0, 0
            summary.append([m, mids[m][0], crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                            right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
    return ['MasterID', 'NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs',
            'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y'], summary

In [161]:
headers, crash_data = intersection_summaries(approved_masters, masters, node_data)


Master 33755 (03/24/2016 - 10/16/2016):
	Nodes [33755, <__main__.Intersection instance at 0x085E5260>, <__main__.Intersection instance at 0x0D0500F8>] 
	12 crashes
	3 ped injuries
	0 bikeinjuries
	3 mvo injuries
	7 right angle crashes


In [32]:
for i in range(5):
    print headers[]


SyntaxError: invalid syntax (<ipython-input-32-e67a03377f2b>, line 2)

In [48]:
print headers[1:14]

['NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs', 'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y']


In [25]:
print approved_masters.keys()

[33755]


In [28]:
print len(approved_masters[33755])

12


x=[1061610523,
1061612065,
1061610582,
1061610996,
1061612065,
1061610582,
1061610996,
1061612065,
1061612261,
1061610326,
1061610582,
1061612206,
1061612537]

y=[1061610117,
1061610251,
1061610286,
1061610293,
1061610326,
1061610334,
1061610343,
1061610389,
1061610431,
1061610523,
1061610582,
1061610741,
1061610840,
1061610948,
1061611237,
1061611268,
1061612131,
1061612261,
1061612291,
1061612306,
1061612384,
1061612537,
1061612652]

for i in x:
    if i not in y: print(i)